# Projeto 1 - Ciência dos Dados

Nome: Giovanni Rozatti

Nome: Gabriel Parfan Guimarães

Atenção: Serão permitidos grupos de três pessoas, mas com uma rubrica mais exigente. Grupos deste tamanho precisarão fazer um questionário de avaliação de trabalho em equipe

___
Carregando algumas bibliotecas:

In [1]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os

In [2]:
print('Esperamos trabalhar no diretório')
print(os.getcwd())

Esperamos trabalhar no diretório
C:\Users\User\OneDrive - Insper - Institudo de Ensino e Pesquisa\Desktop\Insper\2°Semestre DP\CDados\CD2020-Projeto1


Carregando a base de dados com os tweets classificados como relevantes e não relevantes:

In [3]:
#dados = pd.read_excel('PRODUTO.xlsx')

___
## Classificador automático de sentimento


Faça aqui uma descrição do seu produto e o que considerou como relevante ou não relevante na classificação dos tweets.

- Produto escolhido: Cinemark
- Descrição do Produto: A Cinemark USA, Inc. é uma cadeia americana de cinema de propriedade da Cinemark Holdings, Inc., operando nas Américas e em Taiwan. Está sediada em Plano, Texas, na área de Dallas-Fort Worth. É a maior cadeia de cinemas do Brasil, com 30% de participação de mercado.(Wikipedia)
- Método de classificação: 
    - Opiniões positivas: Independentemente do preço, qualidade do serviço, concorrencia, COVID-19 existe uma vontade de visitar o cinema; Também foram classificados como positivos opiniões neutras, pois apesar de não explicitarem opinião positiva (nem negativa), esse foi um tópico escolhido para se falar sobre, o que foi compreendido por mim como algo positivo
    - Opiniões negativas: criticam o Cinemark em algum aspecto, não demonstram interesse em ir ao cinemark, ou mencionam um concorrrente como "tão bom quanto"
    
- Observação: a estrutura do código a seguir é similar ao do usado no meu trabalho no semestre anterior (perfil no git: VonIgnia), e pode ser visto em: `https://github.com/carolinechaim/Projeto_2/blob/master/NaiveBayes teste 1.ipynb` mais especificamente nos commits "Planilha teste rodando no Jupyter" e "Planilha teste rodando os classificadores NaiveBayes".

___
### Montando um Classificador Naive-Bayes

Considerando apenas as mensagens da planilha Treinamento, ensine  seu classificador.

In [4]:
train = pd.read_excel("Cinemark segunda 16-03.xlsx",sheet_name = 'Treinamento', index = False)
#Lê as planilhas de treinamento

In [5]:
probs = train["Opinião"].value_counts(True);
#Calculando as frequências relativas de cada categoria

In [6]:
prob_OP = probs[0] #Opinião Positiva
prob_ON = probs[1] #Opinião Negativa

Separa todas as palavras que aparecem em cada categoria, não levando em conta repetições
(caso uma palavra apareça mais de uma vez em uma mesma categoria, ela será contabilizada apenas uma vez naquela categoria 

In [7]:
ON = train[train["Opinião"] == 0]
ON = ON["Treinamento"].str.cat().split()
series_ON = pd.Series(ON)

In [8]:
OP = train[train["Opinião"] == 1]
OP = OP["Treinamento"].str.cat().split()
series_OP = pd.Series(OP)

Criando uma "tabela" com a probabilidade individual de todas as palavras aparecerem em uma dada categoria
("Dado que a opinião é essa qual a probabilidade dessa palavra aparecer?")

In [9]:
tabela_negativa = series_ON.value_counts(True)
tabela_negativa;

In [10]:
tabela_positiva = series_OP.value_counts(True)
tabela_positiva;

Criando uma função que faz o calculo da frase pertencer a uma categoria utilizando a fórmula de bayes

In [11]:
def classificador_de_palavras_naive_bayes(tweet):
    lista_palavras = tweet.split()
    prob_positivo = 1
    prob_negativo = 1
    for palavra in lista_palavras:
        if palavra in tabela_negativa.index:
            prob_palavra_negativa = tabela_negativa[palavra]
        else:
            prob_palavra_negativa = 1
        if palavra in tabela_positiva.index:
             prob_palavra_positivo = tabela_positiva[palavra]
        else:
             prob_palavra_positivo = 1
        prob_negativo *= prob_palavra_negativa
        prob_positivo *= prob_palavra_positivo
        if (prob_negativo > prob_positivo):
            return "Negativo"
        else:
            return "Positivo"

___
### Verificando a performance do Classificador

Agora você deve testar o seu classificador com a base de Testes.

leitura da planilha para testes com o classificador

In [12]:
test = pd.read_excel("Cinemark segunda 16-03.xlsx",sheet_name = 'Teste', index = False).set_index("Teste")
lista_tweets = list(test.index)

In [13]:
for tweet in lista_tweets:
    test.loc[tweet,"Classificação"] = classificador_de_palavras_naive_bayes(tweet)
test

,Classificação
Teste,
@peresftw vamos apresentar isso no cinemark agora msm,Positivo
cinemark eu queria saber oq q eu te fiz me diz onde foi q eu errei,Positivo
vcs compram no proprio site do cinemark ou no ingresso. com??????,Negativo
@canellamatheus quando lançar vamo todo mundo na locadora de paraibúna hackear um cinemark pra poder ver space jam 2 online gratis 3d,Positivo
é bom que o cinemark tenha um combo bonito de bw hein,Positivo
"@gabrielaberte1 às quartas, o valor é meia para todos (varia de 12 a 16). com um dos benefícios que citei (além também do cartão fidelidade deles, o cinemark mania), você pode pagar meia da meia entrada já vigente, ou seja, sai até por 6, 8 reais a entrada.",Positivo
lembrabdo q eu quase chorei pq n pude comprar o tordo na promocao q teve na cinemark no esperanca parte1acho.... caras eu ia andar c o broche de um lado pro outro,Positivo
"galera, vamos falar sério? aproveitem as festas dessa semana como se fossem as últimas.\nvale lembrar o cinemark já tá fechado, e algumas cidades já estão cancelando seus eventos.\naproveitem enquanto ainda podemos nos dar esse luxo.\ncomo produtora, já estou esperando a bomba.",Positivo
"eu juro que se a cinemark contratasse o joão de novo, eu voltaria a trabalhar lá só por causa dele",Negativo


___
### Concluindo

Versão 1  
22/03- o código roda mas claramente a classificação está com imperfeições isso pode se dar por:
 - erros de contabilização na fórmula do NaiveBayes
 - erros na clasificação do excel
 - certas palavras possuirem um viés deterministico mais forte para um dos lados (no caso a Opinião Positiva)

___
## Aperfeiçoamento:

Os trabalhos vão evoluir em conceito dependendo da quantidade de itens avançados:

* Limpar: \n, :, ", ', (, ), etc SEM remover emojis
* Corrigir separação de espaços entre palavras e emojis ou entre emojis e emojis
* Propor outras limpezas e transformações que não afetem a qualidade da informação ou classificação
* Criar categorias intermediárias de relevância baseadas na probabilidade: ex.: muito relevante, relevante, neutro, irrelevante, muito irrelevante (3 categorias: C, mais categorias conta para B)
* Explicar por que não posso usar o próprio classificador para gerar mais amostras de treinamento
* Propor diferentes cenários para Naïve Bayes fora do contexto do projeto
* Sugerir e explicar melhorias reais com indicações concretas de como implementar (indicar como fazer e indicar material de pesquisa)
* Montar um dashboard que periodicamente realiza análise de sentimento e visualiza estes dados

___
## Referências

[Naive Bayes and Text Classification](https://arxiv.org/pdf/1410.5329.pdf)  **Mais completo**

[A practical explanation of a Naive Bayes Classifier](https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/) **Mais simples**